Supervisor is a multi-agent architecture where specialized agents are coordinated by a central supervisor agent. The supervisor agent controls all communication flow and task delegation, making decisions about which agent to invoke based on the current context and task requirements.

![Supervisor](supervisor-agent.png)

In [19]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
from pprint import pprint
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()
groq_api_key  = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
weather_api = os.getenv("WEATHER_API")
groq_api_key = os.getenv("GROQ_API_KEY")

## langsmith Tracing
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Supervisor Multi Agent"

llm = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct", groq_api_key=groq_api_key)

In [20]:
from langchain_tavily import TavilySearch

web_search = TavilySearch(max_results=3)

In [21]:
import requests
from langchain_core.tools import tool

@tool
def get_weather_data(city: str) -> str:
    """
    This function fetches the current weather data for a given city
    """
    url = f"http://api.weatherstack.com/current?access_key={weather_api}&query={city}"
    response = requests.get(url)
    return response.json()

In [22]:
get_weather_data.invoke(
    {
        "city": "Dhaka"
    }
)

{'request': {'type': 'City',
  'query': 'Dhaka, Bangladesh',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Dhaka',
  'country': 'Bangladesh',
  'region': '',
  'lat': '23.723',
  'lon': '90.409',
  'timezone_id': 'Asia/Dhaka',
  'localtime': '2025-05-16 20:59',
  'localtime_epoch': 1747429140,
  'utc_offset': '6.0'},
 'current': {'observation_time': '02:59 PM',
  'temperature': 29,
  'weather_code': 176,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0025_light_rain_showers_night.png'],
  'weather_descriptions': ['Patchy rain nearby'],
  'astro': {'sunrise': '05:16 AM',
   'sunset': '06:34 PM',
   'moonrise': '10:01 PM',
   'moonset': '07:40 AM',
   'moon_phase': 'Waning Gibbous',
   'moon_illumination': 90},
  'air_quality': {'co': '458.8',
   'no2': '7.215',
   'o3': '123',
   'so2': '12.395',
   'pm2_5': '57.72',
   'pm10': '85.47',
   'us-epa-index': '3',
   'gb-defra-index': '3'},
  'wind_speed': 16,
  'wind_degree': 200,
  'w

## build the Search agent

In [24]:
from langgraph.prebuilt import create_react_agent

research_agent = create_react_agent(
    model=llm,
    tools=[web_search],
    prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="research_agent",
)

In [26]:
response = research_agent.invoke(
    {
        "messages": [HumanMessage(content="Name of the capital of bangladesh")]
    }
)

In [27]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

Name of the capital of bangladesh
================================== Ai Message ==================================
Name: research_agent
Tool Calls:
  tavily_search (call_96cj)
 Call ID: call_96cj
  Args:
    query: capital of bangladesh
    search_depth: basic
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "capital of bangladesh", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "What Is The Capital Of Bangladesh? - WorldAtlas", "url": "https://www.worldatlas.com/articles/what-is-the-capital-of-bangladesh.html", "content": "Learn about the history, geography, and culture of Dhaka, the largest and most populous city in Bangladesh. Find out how Dhaka became the capital of Bangladesh in 1971 and what makes it a unique and diverse city.", "score": 0.8742601, "raw_content": null}, {"title": "Bang

In [37]:
weather_agent = create_react_agent(
    model=llm,
    tools=[get_weather_data],
    prompt=(
        "You are a Weather agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with when need to know actual weather info of any city, DO NOT do any math or other task\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results(weather condition and temperature) of your work, do NOT include ANY other text."
    ),
    name="weather_agent",
)

In [38]:
response = weather_agent.invoke(
    {
        "messages": [HumanMessage(content="Name of the weather condition today in Dhaka?")]
    }
)

In [39]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

Name of the weather condition today in Dhaka?
================================== Ai Message ==================================
Name: weather_agent
Tool Calls:
  get_weather_data (call_q4d4)
 Call ID: call_q4d4
  Args:
    city: Dhaka
================================= Tool Message =================================
Name: get_weather_data

{"request": {"type": "City", "query": "Dhaka, Bangladesh", "language": "en", "unit": "m"}, "location": {"name": "Dhaka", "country": "Bangladesh", "region": "", "lat": "23.723", "lon": "90.409", "timezone_id": "Asia/Dhaka", "localtime": "2025-05-16 21:11", "localtime_epoch": 1747429860, "utc_offset": "6.0"}, "current": {"observation_time": "03:11 PM", "temperature": 29, "weather_code": 113, "weather_icons": ["https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0008_clear_sky_night.png"], "weather_descriptions": ["Clear "], "astro": {"sunrise": "05:16 AM", "s

## Math agent

In [40]:
def add(a: float, b: float):
    """Add two numbers."""
    return a + b


def multiply(a: float, b: float):
    """Multiply two numbers."""
    return a * b


def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b


math_agent = create_react_agent(
    model=llm,
    tools=[add, multiply, divide],
    prompt=(
        "You are a math agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with math-related tasks\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="math_agent",
)

In [43]:
from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model
from langgraph.graph import MessagesState

supervisor = create_supervisor(
    model=llm,
    agents=[research_agent, math_agent, weather_agent],
    prompt=(
        "You are a supervisor managing two agents:\n"
        "- a research agent. Assign research-related tasks to this agent\n"
        "- a math agent. Assign math-related tasks to this agent\n"
        "- a Weather agent Assign to get realtime weather of any city related tasks to this agent\n"
        "Assign work to one agent at a time, do not call agents in parallel.\n"
        "Do not do any work yourself."
    ),
    add_handoff_back_messages=True,
    output_mode="full_history",
).compile()

In [49]:
response = supervisor.invoke(
    {
        "messages": [HumanMessage(content="Find the capital of Bangladesh, then find the its current weather condition today.")]
    }
)


In [50]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

Find the capital of Bangladesh, then find the its current weather condition today.
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_agent (call_ekef)
 Call ID: call_ekef
  Args:
    query: What is the capital of Bangladesh?
================================= Tool Message =================================
Name: transfer_to_research_agent

Successfully transferred to research_agent
================================== Ai Message ==================================
Name: research_agent
Tool Calls:
  tavily_search (call_jtfa)
 Call ID: call_jtfa
  Args:
    query: What is the capital of Bangladesh?
================================= Tool Message =================================
Name: tavily_search

{"query": "What is the capital of Bangladesh?", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "What